In [2]:
import os
import json
import cv2
import numpy as np

import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

import pvlib
from datetime import datetime

# Load Data

## Load roof segment polygons

In [117]:
gdf = gpd.read_file('../[4] Module Fitting/roof_gdf_module_fitting_flatsouth.geojson')
gdf.head()

,index,segment_id,building_id,address,image_id,category_id,num_panels,panel_area,geometry
0,0,90215,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",2986,12,1160,1898.6880,"MULTIPOLYGON (((121.12337 14.66163, 121.12339 ..."
1,1,112141,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",3707,5,888,1453.4784,"POLYGON ((121.12283 14.66167, 121.12283 14.661..."
2,2,53649,way/871081677,"128 Santan, Marikina, Metro Manila, Philippines",1775,13,740,1211.2320,"POLYGON ((121.12561 14.65969, 121.12560 14.659..."
3,3,90212,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",2986,5,734,1201.4112,"POLYGON ((121.12333 14.66191, 121.12333 14.661..."
4,4,19982,way/16827525,"Unit 2128, Riverbank Center, Riverbanks Avenue...",653,3,710,1162.1280,"POLYGON ((121.08306 14.63122, 121.08306 14.631..."


# Get total panel_area and num_panels for each building

In [52]:
marikina_buildings_data  = gpd.read_file('../WebApp Dummy Data/marikina_polygons_with_address.geojson')

In [53]:
marikina_df = marikina_buildings_data[['id','geometry', 'address']]
marikina_df = marikina_df[marikina_df['geometry'].apply(lambda x : x.type!='Point' )]
marikina_df.head()

,id,geometry,address
0,way/4392200,"POLYGON ((121.09815 14.63357, 121.09786 14.633...","J3MX+763, Marikina, 1800 Metro Manila, Philipp..."
1,way/4947814,"POLYGON ((121.08072 14.62996, 121.08078 14.629...","1800 Riverbanks Ave, Marikina, 1800 Metro Mani..."
2,way/4947816,"POLYGON ((121.08178 14.63214, 121.08150 14.631...","300 Riverbanks Ave, Sto. Nino, Marikina, 1800 ..."
3,way/5105906,"POLYGON ((121.11541 14.66517, 121.11576 14.663...","Block 5 Lot 33 C M Recto Santo Nino, Marikina,..."
4,way/5106137,"POLYGON ((121.11880 14.66317, 121.11900 14.663...","109 C.M. Recto St, Marikina, Metro Manila, Phi..."


In [118]:
# groupby building id, and sum the panel area and num_panels
building_panels_df = gdf.groupby(['building_id']).sum().drop(columns=['index', 'segment_id', 'image_id', 'category_id'])
building_panels_df = building_panels_df.reset_index(level='building_id')
building_panels_df = building_panels_df.rename(columns={'building_id': 'id'})
building_panels_df.head()

,id,num_panels,panel_area
0,way/101673369,111,181.6848
1,way/102253416,10375,16981.8000
2,way/102524380,324,530.3232
3,way/102524386,73,119.4864
4,way/102524393,390,638.3520


In [119]:
# add osm bldg polygon to module fitting dataframe
final_building_df = pd.merge(marikina_df, building_panels_df, left_on='id', right_on='id')
final_building_df.head()

,id,geometry,address,num_panels,panel_area
0,way/4392200,"POLYGON ((121.09815 14.63357, 121.09786 14.633...","J3MX+763, Marikina, 1800 Metro Manila, Philipp...",4,6.5472
1,way/4947814,"POLYGON ((121.08072 14.62996, 121.08078 14.629...","1800 Riverbanks Ave, Marikina, 1800 Metro Mani...",3308,5414.5344
2,way/4947816,"POLYGON ((121.08178 14.63214, 121.08150 14.631...","300 Riverbanks Ave, Sto. Nino, Marikina, 1800 ...",15753,25784.5104
3,way/5105906,"POLYGON ((121.11541 14.66517, 121.11576 14.663...","Block 5 Lot 33 C M Recto Santo Nino, Marikina,...",16260,26614.3680
4,way/5106137,"POLYGON ((121.11880 14.66317, 121.11900 14.663...","109 C.M. Recto St, Marikina, Metro Manila, Phi...",241,394.4688


In [120]:
final_building_df.to_csv('building_panel_info_flatsouth.csv')
# final_building_df.to_csv('building_panel_info_flatsw.csv')
# final_building_df.to_csv('building_panel_info_2007_2018_flatsouth.csv')

# Format data for web app format

In [121]:
# load hourly, monthly, and annual data
hourly_data = pd.read_csv('./hourly_solar_potential_df_flatsouth.csv')
monthly_data = pd.read_csv('./monthly_solar_potential_df_flatsouth.csv')
annual_data = pd.read_csv('./annual_solar_potential_df_flatsouth.csv')

# hourly_data = pd.read_csv('./hourly_solar_potential_df_flatsw.csv')
# monthly_data = pd.read_csv('./monthly_solar_potential_df_flatsw.csv')
# annual_data = pd.read_csv('./annual_solar_potential_df_flatsw.csv')

# hourly_data = pd.read_csv('./hourly_solar_potential_df_2007_2018_flatsouth.csv')
# monthly_data = pd.read_csv('./monthly_solar_potential_df_2007_2018_flatsouth.csv')
# annual_data = pd.read_csv('./annual_solar_potential_df_2007_2018_flatsouth.csv')


# load building info data
# final_building_df = pd.read_csv('./building_panel_info_flatsouth.csv')
# final_building_df = gpd.read_file('./building_panel_info_flatsouth.csv')
# final_building_df.crs = 'epsg:4326'

In [122]:
def format_hourly(hourly_data):
  grouped_hourly = hourly_data.groupby(['building_id', 'Hour']).sum().reset_index()
  hourly_formatted = grouped_hourly.pivot(index='building_id', columns='Hour', values='solar_potential')
  hourly_formatted = hourly_formatted.reset_index().rename(columns={0: 'avg_hour_0',
                                                                    1: 'avg_hour_1',
                                                                    2: 'avg_hour_2',
                                                                    3: 'avg_hour_3',
                                                                    4: 'avg_hour_4',
                                                                    5: 'avg_hour_5',
                                                                    6: 'avg_hour_6',
                                                                    7: 'avg_hour_7',
                                                                    8: 'avg_hour_8',
                                                                    9: 'avg_hour_9',
                                                                    10: 'avg_hour_10',
                                                                    11: 'avg_hour_11',
                                                                    12: 'avg_hour_12',
                                                                    13: 'avg_hour_13',
                                                                    14: 'avg_hour_14',
                                                                    15: 'avg_hour_15',
                                                                    16: 'avg_hour_16',
                                                                    17: 'avg_hour_17',
                                                                    18: 'avg_hour_18',
                                                                    19: 'avg_hour_19',
                                                                    20: 'avg_hour_20',
                                                                    21: 'avg_hour_21',
                                                                    22: 'avg_hour_22',
                                                                    23: 'avg_hour_23'
                                                                    })
  return hourly_formatted

def format_monthly(monthly_data):
  grouped_monthly = monthly_data.groupby(['building_id', 'Month']).sum().reset_index()
  monthly_formatted = grouped_monthly.pivot(index='building_id', columns='Month', values='solar_potential')
  monthly_formatted = monthly_formatted.reset_index().rename(columns={1: 'avg_month_1',
                                                                      2: 'avg_month_2',
                                                                      3: 'avg_month_3',
                                                                      4: 'avg_month_4',
                                                                      5: 'avg_month_5',
                                                                      6: 'avg_month_6',
                                                                      7: 'avg_month_7',
                                                                      8: 'avg_month_8',
                                                                      9: 'avg_month_9',
                                                                      10: 'avg_month_10',
                                                                      11: 'avg_month_11',
                                                                      12: 'avg_month_12',})
  return monthly_formatted

def format_yearly(yearly_data):
  grouped_yearly = yearly_data.groupby(['building_id']).sum().reset_index()
  yearly_formatted = grouped_yearly.rename(columns={'solar_potential': 'total_kwh'})
  yearly_formatted = yearly_formatted.drop(columns=['segment_id', 'Unnamed: 0'])

  return yearly_formatted

In [123]:
def format_solar_potential_data(hourly_data, monthly_data, yearly_data, building_panel_info):
  # hourly
  hourly_formatted = format_hourly(hourly_data)
  # monthly
  monthly_formatted = format_monthly(monthly_data)
  # yearly
  yearly_formatted = format_yearly(yearly_data)


  # merge hourly and monthly
  hourly_monthly_data = pd.merge(hourly_formatted, monthly_formatted, left_on='building_id', right_on='building_id')
  # merge with yearly
  final_data = pd.merge(hourly_monthly_data, yearly_formatted, left_on='building_id', right_on='building_id')

  # rename building id to id
  final_data = final_data.rename(columns={'building_id': 'id'})

  # add building info
  final_data = pd.merge(building_panel_info, final_data, left_on='id', right_on='id')

  return final_data

In [124]:
final_valid_data = format_solar_potential_data(hourly_data, monthly_data, annual_data, final_building_df)
final_valid_data.head()

,id,geometry,address,num_panels,panel_area,avg_hour_0,avg_hour_1,avg_hour_2,avg_hour_3,avg_hour_4,...,avg_month_4,avg_month_5,avg_month_6,avg_month_7,avg_month_8,avg_month_9,avg_month_10,avg_month_11,avg_month_12,total_kwh
0,way/4392200,"POLYGON ((121.09815 14.63357, 121.09786 14.633...","J3MX+763, Marikina, 1800 Metro Manila, Philipp...",4,6.5472,0.0,0.0,0.0,0.0,0.0,...,121.799808,120.874923,110.667060,108.223943,100.522987,107.129981,85.413911,85.992027,89.878759,1.282380e+03
1,way/4947814,"POLYGON ((121.08072 14.62996, 121.08078 14.629...","1800 Riverbanks Ave, Marikina, 1800 Metro Mani...",3308,5414.5344,0.0,0.0,0.0,0.0,0.0,...,95478.485625,97969.436917,91088.852380,88415.504533,79503.541618,81989.875234,63640.141696,61632.344911,63619.649196,9.787500e+05
2,way/4947816,"POLYGON ((121.08178 14.63214, 121.08150 14.631...","300 Riverbanks Ave, Sto. Nino, Marikina, 1800 ...",15753,25784.5104,0.0,0.0,0.0,0.0,0.0,...,460651.674194,475842.932919,443030.099559,430179.235536,380449.258310,392011.544671,299924.867572,286914.767522,294396.764333,4.667525e+06
3,way/5105906,"POLYGON ((121.11541 14.66517, 121.11576 14.663...","Block 5 Lot 33 C M Recto Santo Nino, Marikina,...",16260,26614.3680,0.0,0.0,0.0,0.0,0.0,...,468748.131643,479973.344935,445828.798471,432502.108422,389737.225059,401742.393690,312525.900464,303669.018655,313075.886428,4.806828e+06
4,way/5106137,"POLYGON ((121.11880 14.66317, 121.11900 14.663...","109 C.M. Recto St, Marikina, Metro Manila, Phi...",241,394.4688,0.0,0.0,0.0,0.0,0.0,...,7217.232974,7532.154775,7021.139129,6851.704479,5931.394029,6190.946186,4649.227741,4362.122507,4496.389281,7.268944e+04


In [128]:
# add other polygons without data in final df
valid_bldg_ids = final_valid_data['id'].unique()

# bldg df with invalid bldgs
invalid_bldg_df = marikina_df.query("id not in @valid_bldg_ids").copy()      

final_data = final_valid_data.append(invalid_bldg_df)
final_data = final_data.fillna(-999)
final_data

,id,geometry,address,num_panels,panel_area,avg_hour_0,avg_hour_1,avg_hour_2,avg_hour_3,avg_hour_4,...,avg_month_4,avg_month_5,avg_month_6,avg_month_7,avg_month_8,avg_month_9,avg_month_10,avg_month_11,avg_month_12,total_kwh
0,way/4392200,"POLYGON ((121.09815 14.63357, 121.09786 14.633...","J3MX+763, Marikina, 1800 Metro Manila, Philipp...",4.0,6.5472,0.0,0.0,0.0,0.0,0.0,...,121.799808,120.874923,110.667060,108.223943,100.522987,107.129981,85.413911,85.992027,89.878759,1.282380e+03
1,way/4947814,"POLYGON ((121.08072 14.62996, 121.08078 14.629...","1800 Riverbanks Ave, Marikina, 1800 Metro Mani...",3308.0,5414.5344,0.0,0.0,0.0,0.0,0.0,...,95478.485625,97969.436917,91088.852380,88415.504533,79503.541618,81989.875234,63640.141696,61632.344911,63619.649196,9.787500e+05
2,way/4947816,"POLYGON ((121.08178 14.63214, 121.08150 14.631...","300 Riverbanks Ave, Sto. Nino, Marikina, 1800 ...",15753.0,25784.5104,0.0,0.0,0.0,0.0,0.0,...,460651.674194,475842.932919,443030.099559,430179.235536,380449.258310,392011.544671,299924.867572,286914.767522,294396.764333,4.667525e+06
3,way/5105906,"POLYGON ((121.11541 14.66517, 121.11576 14.663...","Block 5 Lot 33 C M Recto Santo Nino, Marikina,...",16260.0,26614.3680,0.0,0.0,0.0,0.0,0.0,...,468748.131643,479973.344935,445828.798471,432502.108422,389737.225059,401742.393690,312525.900464,303669.018655,313075.886428,4.806828e+06
4,way/5106137,"POLYGON ((121.11880 14.66317, 121.11900 14.663...","109 C.M. Recto St, Marikina, Metro Manila, Phi...",241.0,394.4688,0.0,0.0,0.0,0.0,0.0,...,7217.232974,7532.154775,7021.139129,6851.704479,5931.394029,6190.946186,4649.227741,4362.122507,4496.389281,7.268944e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69826,way/930960296,"POLYGON ((121.08218 14.62968, 121.08216 14.629...","1800 Riverbanks Ave, Marikina, 1800 Metro Mani...",-999.0,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-9.990000e+02
69827,way/930960297,"POLYGON ((121.08280 14.63263, 121.08286 14.632...","84 A. Bonifacio Ave, Marikina, 1800 Metro Mani...",-999.0,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-9.990000e+02
69829,way/931231230,"POLYGON ((121.08013 14.62737, 121.08008 14.627...","6 Maj. Santos Dizon St, Marikina, 1800 Metro M...",-999.0,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-9.990000e+02
69832,way/949706391,"POLYGON ((121.09902 14.62061, 121.09913 14.620...","91 Liamzon, Marikina, 1612 Metro Manila, Phili...",-999.0,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-9.990000e+02


In [129]:
final_data.to_file("solar_potential_flatsouth.geojson", driver="GeoJSON")
# final_data.to_file("solar_potential_flatsw.geojson", driver="GeoJSON")
# final_data.to_file("solar_potential_2007_2018_flatsouth.geojson", driver="GeoJSON")